In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
import numpy as np
from tensorflow.keras.models import load_model



In [2]:
# Load the saved features and labels
all_features_vgg16 = np.load('/Users/saikarthik/Desktop/scp/ga_vgg16/all_features_vgg16.npy')
all_labels_vgg16 = np.load('/Users/saikarthik/Desktop/scp/ga_vgg16/all_labels_vgg16.npy')

# Print shape to verify loading
print(f'Features shape: {all_features_vgg16.shape}')
print(f'Labels shape: {all_labels_vgg16.shape}')


Features shape: (6672, 25088)
Labels shape: (6672, 4)


In [3]:
all_features_vgg19 = np.load('/Users/saikarthik/Desktop/scp/ga_vgg19/all_features_vgg19.npy')
all_labels_vgg19 = np.load('/Users/saikarthik/Desktop/scp/ga_vgg19/all_labels_vgg19.npy')

# Print shape to verify loading
print(f'Features shape: {all_features_vgg16.shape}')
print(f'Labels shape: {all_labels_vgg16.shape}')

Features shape: (6672, 25088)
Labels shape: (6672, 4)


In [4]:
from tensorflow.keras.models import load_model

# Load the best VGG16 and VGG19 models
best_vgg16_model = load_model('/Users/saikarthik/Desktop/scp/ga_vgg16/best_vgg16.h5')
best_vgg19_model = load_model('/Users/saikarthik/Desktop/scp/ga_vgg19/best_vgg19.h5')


2024-11-20 06:33:29.731589: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Pro
2024-11-20 06:33:29.731617: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2024-11-20 06:33:29.731631: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2024-11-20 06:33:29.731703: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-11-20 06:33:29.731760: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [5]:
all_features_vgg16

array([[-0.01595471, -0.02323486, -0.03968799, ..., -0.0262275 ,
        -0.0721586 , -0.04747448],
       [-0.01595471, -0.02323486, -0.03968799, ..., -0.0262275 ,
        -0.11433829, -0.04747448],
       [-0.01595471, -0.02323486, -0.03968799, ..., -0.0262275 ,
        -0.59788954, -0.04747448],
       ...,
       [-0.01595471, -0.02323486, -0.03968799, ..., -0.0262275 ,
        -0.37254465, -0.04747448],
       [-0.01595471, -0.02323486, -0.03968799, ..., -0.0262275 ,
        -0.59788954, -0.04747448],
       [-0.01595471, -0.02323486, -0.03968799, ..., -0.0262275 ,
        -0.10403824, -0.04747448]], dtype=float32)

In [6]:
all_features_vgg19

array([[-1.2243470e-02, -2.8450558e-02, -2.8095832e-02, ...,
        -1.6167998e-02, -3.4550823e-02,  1.3985383e+01],
       [-1.2243470e-02, -2.8450558e-02, -2.8095832e-02, ...,
        -1.6167998e-02, -3.4550823e-02, -6.0834702e-02],
       [-1.2243470e-02, -2.8450558e-02, -2.8095832e-02, ...,
        -1.6167998e-02, -3.4550823e-02, -6.0834702e-02],
       ...,
       [-1.2243470e-02, -2.8450558e-02, -2.8095832e-02, ...,
        -1.6167998e-02, -3.4550823e-02, -6.0834702e-02],
       [-1.2243470e-02, -2.8450558e-02, -2.8095832e-02, ...,
        -1.6167998e-02, -3.4550823e-02, -6.0834702e-02],
       [-1.2243470e-02, -2.8450558e-02, -2.8095832e-02, ...,
        -1.6167998e-02, -3.4550823e-02, -6.0834702e-02]], dtype=float32)

In [7]:
all_labels_vgg16

array([[0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       ...,
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.]])

In [8]:
all_labels_vgg19

array([[0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       ...,
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.]])

In [9]:
num_classes = 4

final_predictions = []
sample_index = 0


In [10]:
def normalize_probabilities(predictions):
    return predictions / np.sum(predictions) 


In [11]:
def weighted_average_decision(prob_vgg16, prob_vgg19, weight_vgg16=0.6, weight_vgg19=0.4):
    prob_vgg16 = normalize_probabilities(prob_vgg16)
    prob_vgg19 = normalize_probabilities(prob_vgg19)
    combined_prob = (weight_vgg16 * prob_vgg16) + (weight_vgg19 * prob_vgg19)
    final_class = np.argmax(combined_prob) 
    return final_class

In [12]:
def adjust_weights(pred_vgg16, pred_vgg19, actual_label, weight_vgg16, weight_vgg19, threshold=0.1):
    predicted_class = weighted_average_decision(pred_vgg16, pred_vgg19, weight_vgg16, weight_vgg19)
    if predicted_class != actual_label:
        print("Adjusting weights.")
        
        if np.argmax(pred_vgg16) != actual_label:
            weight_vgg16 = max(0.1, weight_vgg16 - threshold)  
        else:
            weight_vgg16 = min(1.0, weight_vgg16 + threshold)  

        if np.argmax(pred_vgg19) != actual_label:
            weight_vgg19 = max(0.1, weight_vgg19 - threshold) 
        else:
            weight_vgg19 = min(1.0, weight_vgg19 + threshold)  
    
    return weight_vgg16, weight_vgg19, predicted_class


In [13]:
def adjust_weights_with_iterations(pred_vgg16, pred_vgg19, actual_label, weight_vgg16, weight_vgg19, threshold=0.1, max_iterations=5):
    iteration = 0
    predicted_class = weighted_average_decision(pred_vgg16, pred_vgg19, weight_vgg16, weight_vgg19)
    
    while predicted_class != actual_label and iteration < max_iterations:
        print(f"Iteration {iteration + 1}: Prediction is incorrect.")
        weight_vgg16, weight_vgg19, _ = adjust_weights(pred_vgg16, pred_vgg19, actual_label, weight_vgg16, weight_vgg19, threshold)
        
        predicted_class = weighted_average_decision(pred_vgg16, pred_vgg19, weight_vgg16, weight_vgg19)
        
        iteration += 1
    
    return weight_vgg16, weight_vgg19, predicted_class, iteration


In [15]:
# 1 11 6100 1111 634
sample_index =  634
sample_vgg16 = all_features_vgg16[sample_index:sample_index+1] 
sample_vgg19 = all_features_vgg19[sample_index:sample_index+1]  
actual_label = np.argmax(all_labels_vgg16[sample_index])  

weight_vgg16 = 0.6
weight_vgg19 = 0.4

pred_vgg16 = best_vgg16_model.predict(sample_vgg16)  
pred_vgg19 = best_vgg19_model.predict(sample_vgg19)  

predicted_class = weighted_average_decision(pred_vgg16[0], pred_vgg19[0], weight_vgg16, weight_vgg19)



if predicted_class == actual_label :
    
    print(f"Initial prediction: {predicted_class}, Actual label: {actual_label}")
    print("No need to update weights")
else :
    print(f"Initial prediction: {predicted_class}, Actual label: {actual_label}")
    

# Adjust weights and try again
    weight_vgg16, weight_vgg19, predicted_class, iterations = adjust_weights_with_iterations(
    pred_vgg16[0], pred_vgg19[0], actual_label, weight_vgg16, weight_vgg19)

# Print the final result after adjustment
    print(f"Adjusted weights -> VGG16: {weight_vgg16}, VGG19: {weight_vgg19}")
    print(f"Predicted label after {iterations} iteration(s): {predicted_class}")

1/1 [==============================] - 0s 13ms/step
Initial prediction: 2, Actual label: 3
Iteration 1: Prediction is incorrect.
Adjusting weights.
Iteration 2: Prediction is incorrect.
Adjusting weights.
Iteration 3: Prediction is incorrect.
Adjusting weights.
Iteration 4: Prediction is incorrect.
Adjusting weights.
Iteration 5: Prediction is incorrect.
Adjusting weights.
Adjusted weights -> VGG16: 0.10000000000000003, VGG19: 0.1
Predicted label after 5 iteration(s): 2
